In [1]:
import os
import re
import json
import pandas as pd
import psycopg2 as pg2
from time import sleep
from bs4 import BeautifulSoup
from src.crawler import Crawler
from pymongo import MongoClient

brands = [
    'Airline',
    'American Showster',
    'Ampeg',
    'Aria',
    'B.C. Rich',
    'B3',
    'BC Rich',
    'Baldwin',
    'Bilt',
    'Bogner',
    'Burns',
    'Campbell',
    'Charvel',
    'Collings',
    'Conrad',
    'Conradd',
    'Cort',
    "D'Angelico",
    'Danelectro',
    'DeArmond',
    'Dean',
    'DiPinto',
    'Dobro',
    'Duesenberg',
    'EKO',
    'ESP',
    'EVH',
    'Eastman',
    'Eastwood',
    'Electra',
    'Epiphone',
    'Ernie Ball Music Man',
    'Fano',
    'Fender',
    'Framus',
    'G&L',
    'Gibson',
    'Giffin',
    'Godin',
    'Goya',
    'Gretsch',
    'Grosh',
    'Guild',
    'Guyatone',
    'Hagstrom',
    'Hamer',
    'Harden Engineering',
    'Harmony',
    'Heritage',
    'Hofner',
    'Hohner',
    'Ibanez',
    'Ibanez',
    'Intermark',
    'Jackson',
    'James Trussart',
    'Jerry Jones',
    'Kalamazoo',
    'Kay',
    'Kimberly',
    'Kramer',
    'La Baye',
    'Larrivee',
    'Line 6',
    'MCI',
    'Magnatone',
    'Martin',
    'Memphis',
    'Micro-Frets',
    'Moog',
    'Mosrite',
    'Music Man',
    'Musicraft',
    'Musicvox',
    'Nash',
    'National',
    'Norma',
    'Orville',
    'Ovation',
    'Parker',
    'Paul Reed Smith',
    'Peavey',
    'Premier',
    'Reverend',
    'Rickenbacker',
    'Roland',
    'SWR',
    'Schecter',
    'Silvertone',
    'Sorrento',
    'Squier',
    'Steinberger',
    'Stteinbereger',
    'Suhr',
    'Supro',
    'Supro',
    'Taylor',
    'Teisco',
    'The Loar',
    'Tokai',
    'Tom Anderson',
    'Travis',
    'Truetone',
    'Two Rock',
    'Univox',
    'Vox',
    'Washburn',
    'Yamaha'
]

client = MongoClient('192.168.0.211', 27017)
db = client['reverb']
link_coll = db['links']
data_coll = db['data']

In [6]:
df = pd.DataFrame(list(data_coll.find({},{'_id':0})))
df.head()

,title,date,cond,price
0,Rickenbacker 660-12 Tom Petty Signature 1992,2/23/2021,Excellent,"$6,495"
1,Rickenbacker 660-12 Tom Petty Signature 1992,12/7/2020,Excellent,"$4,325"
2,Rickenbacker 660-12 Tom Petty Signature 1992,10/20/2020,Excellent,"$7,995"
3,Rickenbacker 660-12 Tom Petty Signature 1992,9/28/2020,Excellent,"$5,495"
4,Rickenbacker 660-12 Tom Petty Signature 1992,5/26/2020,Excellent,"$5,800"


In [4]:
'|'.join(brands)

"Airline|American Showster|Ampeg|Aria|B.C. Rich|B3|BC Rich|Baldwin|Bilt|Bogner|Burns|Campbell|Charvel|Collings|Conrad|Conradd|Cort|D'Angelico|Danelectro|DeArmond|Dean|DiPinto|Dobro|Duesenberg|EKO|ESP|EVH|Eastman|Eastwood|Electra|Epiphone|Ernie Ball Music Man|Fano|Fender|Framus|G&L|Gibson|Giffin|Godin|Goya|Gretsch|Grosh|Guild|Guyatone|Hagstrom|Hamer|Harden Engineering|Harmony|Heritage|Hofner|Hohner|Ibanez|Ibanez|Intermark|Jackson|James Trussart|Jerry Jones|Kalamazoo|Kay|Kimberly|Kramer|La Baye|Larrivee|Line 6|MCI|Magnatone|Martin|Memphis|Micro-Frets|Moog|Mosrite|Music Man|Musicraft|Musicvox|Nash|National|Norma|Orville|Ovation|Parker|Paul Reed Smith|Peavey|Premier|Reverend|Rickenbacker|Roland|SWR|Schecter|Silvertone|Sorrento|Squier|Steinberger|Stteinbereger|Suhr|Supro|Supro|Taylor|Teisco|The Loar|Tokai|Tom Anderson|Travis|Truetone|Two Rock|Univox|Vox|Washburn|Yamaha"

In [5]:
r1 = ")(?:(?:(.+)((?:Early|Mid|Late)-?\s?'?\d+s"
r2 = "\d{4}\s?-\s?\d{4})(.*))"
r3 = "(?:(.+)(\d{4}s?)(.*))"
r4 = "(?:(.+)(\d{2}s?)(.*)))$"
regex = r"(" + '|'.join(brands) + '|'.join([r1, r2, r3, r4])

for record in df['title'].unique():
    r = re.match(regex, record, re.IGNORECASE)
    try:
        r.groups()
    except:
        print(record)

Fender Kurt Cobain Jaguar Sunburst
Paul Reed Smith SE One Natural
Fender Custom (Maverick)
Gibson Les Paul Slash Signature Appetite Amber
Taylor T3/B Quilted Tobacco Sunburst
Epiphone Matt Heafy Signature 7-String Les Paul Custom
Agile AL-3001
Fender Deluxe Powerhouse Stratocaster
Fender American Standard FSR Rustic Ash Stratocaster
Electric Loog II Mini Guitar Solid Finish
Electric Loog II Mini Guitar Clear Lucite Finish
Fender J5 Triple Tele Deluxe John 5 Signature Telecaster
Ernie Ball Music Man St. Vincent Signature Blue
Ernie Ball Music Man St. Vincent Signature Black


In [4]:
# conn = pg2.connect(dbname='reverb', host='192.168.0.209', password='galvanize', user='postgres')
# cur = conn.cursor()
# conn.autocommit = True
# cur.execute("""CREATE TABLE guitars(
#                id INT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
#                title VARCHAR(225),
#                date_str VARCHAR(12),
#                cond VARCHAR(12),
#                sale_price NUMERIC
#            );""")
# conn.close()